# Importações

In [ ]:
import numpy as np 
import pandas as pd 
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
# Importando o dataset
dsPrincipal = pd.read_csv("athlete_events.csv")

# Normalizando os dados

In [ ]:
# Preenchendo valores nulos em Height e Weight com a média
dsPrincipal['Height'].fillna(dsPrincipal['Height'].mean())
dsPrincipal['Weight'].fillna(dsPrincipal['Weight'].mean())


dsPrincipal['Medal'].fillna("Nenhuma") # Substituindo valores nulos em Medal por "Nenhuma"

In [ ]:

duplicatas = dsPrincipal[dsPrincipal.duplicated()] # Verificar duplicatas

print(duplicatas) # Só para verificar se existe duplicados

dsPrincipal.drop_duplicates(inplace=True) # Remover duplicatas

* Quantos atletas masculinos e femininos participaram em cada olimpíada

In [ ]:
df_unique = dsPrincipal.drop_duplicates(subset=['ID', 'Year']) # Filtrar para garantir que cada atleta é contado apenas uma vez por ano

participacao_por_ano = df_unique.groupby(['Year', 'Sex']).size().unstack(fill_value=0) # Contando o número de atletas

participacao_por_ano.columns = ['Feminino', 'Masculino'] # Trocando o F e M para facilitar a apresentação

print(participacao_por_ano) # Apresentando atletas

* Crie um gráfico em Python e Matplotlib para exibir esses valores, escolhendo a forma mais efetiva de expor a informação.

In [ ]:
anos = participacao_por_ano.index.astype(str)
x = np.arange(len(anos))

width = 0.35 # Definido um width menor para as barras 


plt.figure(figsize=(12, 8)) # Criar o gráfico

plt.bar(x - 0.2/2, participacao_por_ano['Masculino'], width=0.2, label='Masculino', color='blue')
plt.bar(x + 0.2/2, participacao_por_ano['Feminino'], width=0.2 ,label='Feminino', color='red')

# legandas
plt.title("Número de Atletas Masculinos e Femininos por Olimpíada")
plt.xlabel("Ano")
plt.ylabel("Número de Atletas")
plt.legend()

plt.xticks(x, anos, rotation=45)
plt.tight_layout()

plt.show() # Exibir o gráfico

Análise 1. As mulheres conheça a ganhar espaço no esporte. Provavelmente isso aconteceu em decorrência da quebra de tabus relacionados a preconceito de genero.

Análise 2. A segunda guerra mundial impossibilitou os jogos olímpicos

* Quais os 10 atletas com o maior número de medalhas em cada um dos jogos olímpicos.

In [ ]:

medalhas = dsPrincipal.dropna(subset=['Medal']) # Filtrar para considerar apenas os atletas que ganharam medalhas (excluindo NA)

medalhas_por_atleta = medalhas.groupby(['Year', 'ID', 'Name', 'Team'])['Medal'].count().reset_index()

top10_atletas_ano = medalhas_por_atleta.sort_values(['Year', 'Medal'], ascending=[True, False]).groupby('Year').head(10)

top10_atletas_ano # Exibir os resultados

In [ ]:
# Criar um gráfico individual para cada ano (Small Multiples)
anos = top10_atletas_ano['Year'].unique()
fig, axes = plt.subplots(len(anos), 1, figsize=(10, len(anos) * 4), sharex=True)

# Loop para criar gráficos de barras horizontais para cada ano
for i, ano in enumerate(anos):
    top10_atletas = top10_atletas_ano[top10_atletas_ano['Year'] == ano]
    ax = axes[i]
    bars = ax.barh(top10_atletas['Name'] + ' (' + top10_atletas['Team'] + ')', top10_atletas['Medal'], color='skyblue')
    ax.set_title(f'Top 10 Atletas com mais Medalhas - {ano}')
    ax.set_xlabel('Número de Medalhas')
    ax.set_ylabel('Atleta')
    for bar in bars:
        ax.text(
            bar.get_width(),  # Posição X do texto (no final da barra)
            bar.get_y() + bar.get_height() / 2,  # Posição Y do texto (meio da barra)
            f'{int(bar.get_width())}',  # Texto exibido (número de medalhas)
            va='center',  # Alinhamento vertical
            ha='left'  # Alinhamento horizontal
        )

plt.tight_layout()
plt.show()

Análise 1. Os atletas americanos apresentaram um grande desempenho em praticamente todos as edições dos jogos.

* Qual time possui o maior número de medalhas de Ouro? E de prata? E de bronze?

In [ ]:

medalhas_por_time = dsPrincipal.groupby(['Team', 'Medal']).size().unstack(fill_value=0) # Pega o número de medalhas de cada tipo por time

# Identificar o tipo de medalhas por time. 
maior_ouro = medalhas_por_time['Gold'].idxmax(), int (medalhas_por_time['Gold'].max()) # Obs: Passando para inteiro para melhorar a visualização
maior_prata = medalhas_por_time['Silver'].idxmax(), int(medalhas_por_time['Silver'].max())
maior_bronze = medalhas_por_time['Bronze'].idxmax(), int(medalhas_por_time['Bronze'].max())

# Apresenta os valores
print('Ouro', maior_ouro)
print('Prata', maior_prata)
print('Bronze', maior_bronze)

In [ ]:

# Criar um DataFrame com esses valores para facilitar a plotagem
dados_medalhas = pd.DataFrame({
    'Medalha': ['Ouro', 'Prata', 'Bronze'],
    'Time': [maior_ouro[0], maior_prata[0], maior_bronze[0]],
    'Quantidade': [maior_ouro[1], maior_prata[1], maior_bronze[1]],
})

# Configuração do gráfico de barras verticais
plt.figure(figsize=(8, 6))
plt.bar(dados_medalhas['Medalha'], dados_medalhas['Quantidade'], color=['#FFD700', '#C0C0C0', '#CD7F32'])

# Títulos e rótulos
plt.title("Número de Medalhas de Ouro, Prata e Bronze dos Times com Maior Quantidade")
plt.xlabel("Tipo de Medalha")
plt.ylabel("Quantidade de Medalhas")

# Rótulos para cada barra com o nome do time
for i, (medalha, quantidade, time) in enumerate(zip(dados_medalhas['Medalha'], dados_medalhas['Quantidade'], dados_medalhas['Time'])):
    plt.text(i, quantidade + 5, f"{time}\n{quantidade}", ha='center', va='bottom')

plt.show()

Análise 1. Os Estados Unidos fazem grandes investimentos em esportes, infraestrutura, treinamento e desenvolvimento de atletas. Podemos observar na quantidade de premiações para edição dos jogos.

* Para os esportes atletismo, ginástica olímpica e natação, selecione 3 países, sendo um deles o Brasil, e verifique quantas medalhas de ouro, prata e bronze cada um desses países ganharam em cada um desses esportes.

In [ ]:
# Esportes e países de interesse
esportes_interesse = ["Athletics", "Gymnastics", "Swimming"]
paises_interesse = ["Brazil", "United States", "Russia"]

# Considera apenas os esportes e países de interesse
df_esportes = dsPrincipal[
    (dsPrincipal['Sport'].isin(esportes_interesse)) & 
    (dsPrincipal['Team'].isin(paises_interesse))
]


medalhas_por_esporte_pais = df_esportes.groupby(['Team', 'Sport', 'Medal']).size().unstack(fill_value=0) # Contar o número de medalhas

medalhas_por_esporte_pais

* Crie um gráfico em Python e Matplotlib para exibir esses valores, escolhendo a forma mais efetiva de expor a informação.

In [ ]:
medalhas_por_esporte_pais = medalhas_por_esporte_pais[['Gold', 'Silver', 'Bronze']] # Sequeciamento das medalhas

fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(15, 6), sharey=True)
fig.suptitle("Medalhas de Ouro, Prata e Bronze por País e Esporte")


cores = ['#FFD700', '#C0C0C0', '#CD7F32'] # Lista de cores para cada tipo de medalha

# Criar uma barra para cada esporte
for idx, esporte in enumerate(esportes_interesse):
    ax = axes[idx]
    dados_esporte = medalhas_por_esporte_pais.xs(esporte, level="Sport", drop_level=False)
    dados_esporte.plot(kind='bar', stacked=True, color=cores, ax=ax)
    ax.set_title(esporte)
    ax.set_xlabel("País")
    ax.set_ylabel("Número de Medalhas" if idx == 0 else "")
    ax.legend(title="Medalha", loc="upper left")

plt.tight_layout(rect=[0, 0, 1, 0.95])
plt.show()

Analise 1. Podemos observar que o Brasil não é muito expressivo nas modalidades por não ter uma tradição nas categorias.

* Crie dois gráficos em Python e Matplotlib para exibir esses mesmos valores, escolhendo a forma mais efetiva de expor a informação, porém crie um gráfico para os medalhistas masculinos e outro para os femininos.

In [ ]:
# Dividir os dados por sexo
df_masculino = df_esportes[df_esportes['Sex'] == 'M']
df_feminino = df_esportes[df_esportes['Sex'] == 'F']

# Função para plotar o gráfico de medalhas por gênero
def plot_medalhas_por_genero(df, titulo):
    medalhas_por_esporte_pais = df.groupby(['Team', 'Sport', 'Medal']).size().unstack(fill_value=0)
    medalhas_por_esporte_pais = medalhas_por_esporte_pais[['Gold', 'Silver', 'Bronze']]  # Ordenar colunas
    
    fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(15, 6), sharey=True)
    fig.suptitle(titulo)

    cores = ['#FFD700', '#C0C0C0', '#CD7F32']  # Cores para ouro, prata e bronze

    # Criar uma barra para cada esporte
    for idx, esporte in enumerate(esportes_interesse):
        ax = axes[idx]
        dados_esporte = medalhas_por_esporte_pais.xs(esporte, level="Sport", drop_level=False)
        dados_esporte.plot(kind='bar', stacked=True, color=cores, ax=ax)
        ax.set_title(esporte)
        ax.set_xlabel("País")
        ax.set_ylabel("Número de Medalhas" if idx == 0 else "")
        ax.legend(title="Medalha", loc="upper right")

    plt.tight_layout(rect=[0, 0, 1, 0.95])
    plt.show()

# Plotar gráficos para masculino e feminino
plot_medalhas_por_genero(df_masculino, "Medalhas por País e Esporte - Masculino")
plot_medalhas_por_genero(df_feminino, "Medalhas por País e Esporte - Feminino")

Analise 1. A participação na ginastica artistica é majoritariamente feminina, possivelmente devido a estoriotipos acessociados ao esporte, fazendo a participação masculina ser menor. 